# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils # helpers for drawing
mp_pose = mp.solutions.pose

In [3]:
VIDEO_PATH = 'bicep-curl.mp4'
EXPORT_PATH = 'bicep-curl.csv'
MODEL_PATH = 'bicep-curl.pkl'

# X. Make Some Detections

In [4]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
        
        cv2.imshow('LUCIO', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

# 1. Save Video

In [5]:
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('bicep-curl.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()

    try:
        cv2.imshow('Bicep-Curl', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV

In [4]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [5]:
landmarks = ['class']
for val in range(1, 33 +1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']

In [6]:
landmarks[:]

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [58]:
with open('coords.csv', mode= 'w', newline= '') as f:
    csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [59]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        #keypoints = np.insert(keypoints, 0, action)
        #print('normal mid execution')
        with open('coords.csv', mode= 'a', newline= '') as f:
            csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [60]:
cap = cv2.VideoCapture('bicep-curl.mp4')

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    while cap.isOpened():
        ret, image = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        ## results = pose.process(image)
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
                
        k = cv2.waitKey(1)
        if k == ord('a'):
            export_landmark(results, 'correct')
        elif k == ord('s'):
            export_landmark(results, 'error1')
        elif k ==ord('d'):
            export_landmark(results, 'error2')


        cv2.imshow('Bicep Training', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 

In [1]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('coords.csv')

In [9]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.394216,0.823304,0.521883,0.999915,0.400480,0.832445,0.564747,0.999955,0.400534,...,-0.569941,0.905392,0.366447,0.239952,-0.529588,0.682359,0.366910,0.228794,-0.682824,0.893134
1,correct,0.378416,0.808986,0.500686,0.999824,0.388844,0.823843,0.544244,0.999917,0.389674,...,-0.469141,0.857716,0.349367,0.251711,-0.477883,0.606852,0.365400,0.247318,-0.577988,0.830602
2,correct,0.370498,0.800588,0.480671,0.999774,0.382954,0.818234,0.529139,0.999899,0.384757,...,-0.444559,0.837803,0.345535,0.257974,-0.451070,0.593900,0.364240,0.254299,-0.564721,0.809484
3,correct,0.391180,0.809407,0.553187,0.999752,0.394833,0.820088,0.587577,0.999887,0.395430,...,-0.364498,0.823966,0.346334,0.261834,-0.462412,0.603115,0.384745,0.262284,-0.452924,0.788278
4,correct,0.419454,0.826854,0.647878,0.999715,0.419590,0.834117,0.642996,0.999859,0.420833,...,-0.288368,0.806630,0.346707,0.259216,-0.518287,0.624147,0.386834,0.265655,-0.368683,0.762289


In [11]:
df.shape

(1409, 133)

In [12]:
df[df['class'] == 'correct']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.394216,0.823304,0.521883,0.999915,0.400480,0.832445,0.564747,0.999955,0.400534,...,-0.569941,0.905392,0.366447,0.239952,-0.529588,0.682359,0.366910,0.228794,-0.682824,0.893134
1,correct,0.378416,0.808986,0.500686,0.999824,0.388844,0.823843,0.544244,0.999917,0.389674,...,-0.469141,0.857716,0.349367,0.251711,-0.477883,0.606852,0.365400,0.247318,-0.577988,0.830602
2,correct,0.370498,0.800588,0.480671,0.999774,0.382954,0.818234,0.529139,0.999899,0.384757,...,-0.444559,0.837803,0.345535,0.257974,-0.451070,0.593900,0.364240,0.254299,-0.564721,0.809484
3,correct,0.391180,0.809407,0.553187,0.999752,0.394833,0.820088,0.587577,0.999887,0.395430,...,-0.364498,0.823966,0.346334,0.261834,-0.462412,0.603115,0.384745,0.262284,-0.452924,0.788278
4,correct,0.419454,0.826854,0.647878,0.999715,0.419590,0.834117,0.642996,0.999859,0.420833,...,-0.288368,0.806630,0.346707,0.259216,-0.518287,0.624147,0.386834,0.265655,-0.368683,0.762289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,correct,0.330085,0.699986,-0.145416,0.997420,0.343068,0.721529,-0.112593,0.997978,0.345303,...,0.125256,0.379936,0.475425,-0.171171,0.599815,0.100177,0.403914,-0.206719,-0.051390,0.199322
535,correct,0.326041,0.694155,-0.125379,0.997486,0.337568,0.716694,-0.095789,0.998053,0.339866,...,0.075732,0.380614,0.492334,-0.196848,0.499532,0.099291,0.421936,-0.221657,-0.101301,0.200032
536,correct,0.328769,0.687269,-0.100638,0.997446,0.338549,0.709820,-0.072638,0.998046,0.340206,...,0.091619,0.375429,0.491100,-0.211724,0.507108,0.099538,0.420791,-0.236451,-0.098774,0.202052
537,correct,0.326496,0.680291,-0.051574,0.997505,0.335151,0.702815,-0.023659,0.998079,0.336878,...,0.097269,0.378330,0.494188,-0.209387,0.537105,0.099473,0.419598,-0.241676,-0.080947,0.205868


In [13]:
X = df.drop('class', axis= 1) 
y = df['class']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 7)

In [15]:
y_test

1083     error2
313     correct
734      error1
284     correct
671      error1
         ...   
619      error1
618      error1
816      error1
997      error2
779      error1
Name: class, Length: 423, dtype: object

## 3.2 Train Machine Learning Classification Model

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [17]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [18]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [19]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [20]:
fit_models['rc'].predict(X_test)

array(['error2', 'correct', 'error1', 'correct', 'error1', 'error1',
       'error1', 'correct', 'error2', 'error2', 'correct', 'correct',
       'correct', 'error1', 'error2', 'correct', 'error2', 'error2',
       'error1', 'correct', 'correct', 'correct', 'error2', 'correct',
       'error1', 'correct', 'error2', 'correct', 'correct', 'error2',
       'error1', 'error2', 'error2', 'correct', 'correct', 'correct',
       'error2', 'correct', 'correct', 'error1', 'correct', 'error1',
       'error1', 'correct', 'correct', 'error2', 'correct', 'error1',
       'error2', 'error2', 'correct', 'error2', 'error2', 'error2',
       'correct', 'error2', 'correct', 'error1', 'error2', 'error2',
       'error2', 'correct', 'error1', 'correct', 'error2', 'correct',
       'error2', 'correct', 'error1', 'correct', 'error1', 'error2',
       'error1', 'error1', 'correct', 'correct', 'correct', 'correct',
       'correct', 'error2', 'correct', 'correct', 'error2', 'correct',
       'error2', 'error

## 3.3 Evaluate and Serialize Model

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [22]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,
         accuracy_score(y_test.values, yhat),
         precision_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct")
        )

lr 1.0 1.0 1.0 1.0
rc 0.9976359338061466 0.9976359338061466 0.9976359338061466 0.9976359338061466
rf 1.0 1.0 1.0 1.0
gb 0.9929078014184397 0.9929078014184397 0.9929078014184397 0.9929078014184397


c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.war

In [23]:
yhat = fit_models['rf'].predict(X_test)

In [24]:
yhat[ :10]

array(['error2', 'correct', 'error1', 'correct', 'error1', 'error1',
       'error1', 'correct', 'error2', 'error2'], dtype=object)

In [25]:
with open('bicep-curl.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [26]:
with open('bicep-curl.pkl', 'rb') as f:
    pickle.load(f)

In [ ]:
# X = pd.DataFrame([row], columns= landmarks[1:])

In [29]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
state = ''

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    while cap.isOpened():
        ret, image = cap.read()
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
                
        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row], columns= landmarks[1:])
            
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)
            correct = False; error1 = False; error2 = False

            if body_language_class == 'correct' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = True
                error1 = False
                error2 = False
            elif body_language_class == 'error1' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = True
                error2 = False
            elif body_language_class == 'error2' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = False
                error2 = True
            
            if (error1):
                state = 'Elbow is out of place'
            elif (correct):
                state = 'Correct'
            elif(error2):
                state = 'Leaning'

            # elif current_stage = 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= .7:
            #     current_stage = 'up'
            #     counter += 1
            
            counter = 0
            
            
                # print(current_stage)

            # Get status box
            #cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            # Display Class
            cv2.putText(image, 
                        'CLASS',
                        (0,0),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            
            # cv2.putText(image, 
            #             body_language_class(' ')[0],
            #             (90,40),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,
            #             (255,255,255),
            #             2,
            #             cv2.LINE_AA
            #            )
            #Display Propability
            cv2.putText(image, 
                        'PROB',
                        (15,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(round(body_language_prob[np.argmax(body_language_prob)], 2)),
                        (10,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255,255,255),
                        2,
                        cv2.LINE_AA
                       )
            
            #Display Count
            cv2.putText(image, 
                        'COUNT',
                        (180,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(state),
                        (175,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0,0,255),
                        2,
                        cv2.LINE_AA
                       )
            
            
        except Exception as e:
            pass


        cv2.imshow('Lucio', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()